In [1]:
from sqlalchemy.orm import scoped_session, sessionmaker

# Create a session that flushes before each query but never commits.
# Also wrap it in a scoped_session to ensure we use different sessions for different threads.
DBSession = scoped_session(sessionmaker(
    autoflush=True, autocommit=False
))

In [2]:
from sqlalchemy.ext.declarative import declarative_base

# Create a declarative base class from which our models can inherit
# Also takes charge of creating a database metadata object for us.
DeclarativeBase = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, DateTime, Unicode
from datetime import datetime

class Person(DeclarativeBase):
    __tablename__ = 'persons'
    
    uid = Column(Integer, primary_key=True)
    name = Column(Unicode(255), nullable=False)
    
    def __repr__(self):
        return '<Person: %s>' % self.name.encode('ascii', errors='ignore')

In [4]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Log(DeclarativeBase):
    __tablename__ = 'logs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    # Declare ForeignKey on database that stores relation to the Person
    # that owns this Log
    person_id = Column(Integer, ForeignKey(Person.uid))
    
    # Declare a relationship Log.person -> Person 
    # with a backreference Person.logs -> Log[]
    person = relationship(Person, backref="logs")
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person and self.person.name.encode('ascii', errors='ignore'))

In [5]:
from sqlalchemy import create_engine

# Create a connection to an in-memory database
engine = create_engine('sqlite:///:memory:')

# And bind the session to it, so that each action we perform happens on that database.
DBSession.configure(bind=engine)

In [6]:
# Create all the tables we defined so far.
DeclarativeBase.metadata.create_all(engine)

In [7]:
p = Person(name=u'user')
DBSession.add(p)
DBSession.commit()

In [8]:
user = DBSession.query(Person).first()
print user

<Person: user>


In [9]:
l = Log(person=user)
DBSession.add(l)
DBSession.commit()

In [10]:
log = DBSession.query(Log).first()
print log

<Log: 2015-10-16 11:55:08.150996, user>


In [11]:
# Delete the user owning the Log we just created.
DBSession.delete(user)
DBSession.commit()

In [12]:
# As we didn't specify any particular option
# When a member of a relation is deleted, SQLAlchemy 
# automatically propagates the change and updates the related entities
# setting the foreign keys to NULL
print log

<Log: 2015-10-16 11:55:08.150996, None>


In [13]:
class OtherLog(DeclarativeBase):
    __tablename__ = 'otherlogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    # New model which declares the foreign key as not nullable.
    # If we have a Log it must belong to an user.
    person_id = Column(Integer, ForeignKey(Person.uid), nullable=False)
    person = relationship(Person, backref="otherlogs")
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person.name.encode('ascii', errors='ignore'))

DeclarativeBase.metadata.create_all(engine)

In [14]:
p = Person(name=u'user')
DBSession.add(p)

olog = OtherLog(person=p)
DBSession.add(olog)

DBSession.commit()

In [15]:
print olog

<Log: 2015-10-16 11:57:45.106950, user>


In [16]:
DBSession.delete(p)

try:
    # When we flush the session (and commit transaction)
    # SQLAlchemy will try to set the related foreign keys to NULL
    # like before, but in this case it will fail as the ForeignKey
    # cannot be NULL.
    DBSession.commit()
except Exception as e:
    DBSession.rollback()
    print e

(sqlite3.IntegrityError) NOT NULL constraint failed: otherlogs.person_id [SQL: u'UPDATE otherlogs SET person_id=? WHERE otherlogs.uid = ?'] [parameters: (None, 1)]


In [18]:
# So we manually delete the objects before deleting the Person
# so that it won't fail anymore
DBSession.query(OtherLog).delete()
DBSession.delete(p)
DBSession.commit()

In [19]:
from sqlalchemy.orm import backref


class MoreLog(DeclarativeBase):
    __tablename__ = 'morelogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    person_id = Column(Integer, ForeignKey(Person.uid), nullable=False)
    
    # As we want to avoid having to manually delete the related objects
    # we can instruct SQLAlchemy to do that for use by using the
    # cascade='all, delete-orphan' option. As we specified the cascade
    # on the Person->MoreLog backref when the person gets deleted each
    # MoreLog associated to it will be deleted too.
    #
    # PAY ATTENTION TO WHERE YOU SPECIFY THE CASCADE.
    person = relationship(Person, backref=backref("morelogs", cascade='all, delete-orphan'))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person.name.encode('ascii', errors='ignore'))

DeclarativeBase.metadata.create_all(engine)

In [20]:
p = Person(name=u'user')
DBSession.add(p)

mlog = MoreLog(person=p)
DBSession.add(mlog)

DBSession.commit()

In [21]:
# Here are our logs related to the user 
print DBSession.query(MoreLog).all()

[<Log: 2015-10-16 12:02:25.735881, user>]


In [22]:
DBSession.delete(p)
DBSession.commit()

In [23]:
# When we delete the Person owning the MoreLogs they will disappear too
print DBSession.query(MoreLog).all()

[]


In [24]:
# We put back data in place to see what happens
# when we try the opposite.
p = Person(name=u'user')
DBSession.add(p)

mlog = MoreLog(person=p)
DBSession.add(mlog)

DBSession.commit()

In [25]:
mlog = DBSession.query(MoreLog).first()

DBSession.delete(mlog)
DBSession.commit()

In [26]:
# See that our log got actually deleted
print DBSession.query(MoreLog).all()

[]


In [27]:
# Even though we deletedthe MoreLog, the person is still there
print DBSession.query(Person).all()

[<Person: user>]


In [28]:
from sqlalchemy.orm import backref


class OneLog(DeclarativeBase):
    __tablename__ = 'onelogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    person_id = Column(Integer, ForeignKey(Person.uid), nullable=False)
    
    # In case a person can have only one log
    # we can use the uselist=False option to declarete a 1to1 relation.
    person = relationship(Person, backref=backref("log", uselist=False, cascade='all, delete-orphan'))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person.name.encode('ascii', errors='ignore'))

DeclarativeBase.metadata.create_all(engine)

In [29]:
p = Person(name=u'user')
DBSession.add(p)

olog = OneLog(person=p)
DBSession.add(olog)

DBSession.commit()

In [30]:
# Our logs are there with their owner
print DBSession.query(OneLog).all()

[<Log: 2015-10-16 12:05:41.671112, user>]


In [31]:
# And the owner of a log is available through the .person
# attribute, which now is a single Person instead of a list
olog.person

<Person: user>

In [32]:
# Delete the person and it will properly update like before
DBSession.delete(p)
DBSession.commit()

In [33]:
# See our log got deleted too
print DBSession.query(OneLog).all()

[]


In [34]:
# We create all data back to showcase a corner-case
p = Person(name=u'user')
DBSession.add(p)

olog = OneLog(person=p)
DBSession.add(olog)

DBSession.commit()

In [35]:
# relationship propagation only works when we
# use the DBSession.delete() if we issue a
# DELETE statemente, SQLA won't be able to track
# the objects that need to be updated or deleted.
DBSession.query(Person).filter_by(name=u'user').delete()

2

In [38]:
# See that even though we deleted the person, the log is still there
# as we used a DELETE statement instead of DBSession.delete()
l = DBSession.query(OneLog).first()
print l.timestamp

# But it has no person
print l.person

2015-10-16 12:07:18.837266
None


In [39]:
from sqlalchemy import Table
from sqlalchemy.orm import backref


# Let's see what happens with Many2Many.
manylog_person_table = Table('manylog_person', DeclarativeBase.metadata,
   Column('person_id', Integer,
          ForeignKey('persons.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True),
   Column('manylog_id', Integer,
          ForeignKey('manylogs.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True)
)



class ManyLog(DeclarativeBase):
    __tablename__ = 'manylogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    # many2many are declared like all other relations but require
    # a secondary= utility table.
    persons = relationship(Person, secondary=manylog_person_table, 
                           backref=backref("manylogs"))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, ','.join(p.name.encode('ascii', errors='ignore') for p in self.persons))

    
DeclarativeBase.metadata.create_all(engine)

In [40]:
# Create a log with two persons
person1 = Person(name=u'person1')
person2 = Person(name=u'person2')
DBSession.add(person1)
DBSession.add(person2)

mlog = ManyLog(persons=[person1, person2])
DBSession.add(mlog)

DBSession.commit()

In [41]:
# Check the created log refers to the two people
print DBSession.query(ManyLog).all()

[<Log: 2015-10-16 12:10:34.062655, person1,person2>]


In [42]:
# When we delete a person the Log is updated too automatically
p = DBSession.query(Person).filter_by(name='person1').first()
DBSession.delete(p)
DBSession.commit()

/Users/amol/venv/notebook/lib/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:185: SAWarning: Unicode type received non-unicode bind param value 'person1'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))


In [43]:
# Check that our ManyLog now points to only 1 person
print DBSession.query(ManyLog).all()

[<Log: 2015-10-16 12:10:34.062655, person2>]


In [44]:
# Let's see what happens when we delete the second person too.
p = DBSession.query(Person).filter_by(name='person2').first()
DBSession.delete(p)
DBSession.commit()

/Users/amol/venv/notebook/lib/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:185: SAWarning: Unicode type received non-unicode bind param value 'person2'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))


In [45]:
# Our utility table gets automatically updated and the rows got deleted
# No need to specifiy any particular option
DBSession.query(manylog_person_table).all()

[]

In [46]:
# But we don't hve cascading of the entities themselves.
# even though we deleted all the Person pointing to the ManyLog
# the manylog itself is still there.
print DBSession.query(ManyLog).all()

[<Log: 2015-10-16 12:10:34.062655, >]


In [47]:
from sqlalchemy import Table
from sqlalchemy.orm import backref


manylogdel_person_table = Table('manylogdel_person', DeclarativeBase.metadata,
   Column('person_id', Integer,
          ForeignKey('persons.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True),
   Column('manylogdel_id', Integer,
          ForeignKey('manylogsdel.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True)
)



class ManyLogDeletable(DeclarativeBase):
    __tablename__ = 'manylogsdel'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    # Let's try to use the cascade='all, delete-orphan' option to force deletion
    # of our ManyLog when all the person owning it gets deleted.
    persons = relationship(Person, secondary=manylogdel_person_table, backref=backref("manylogsdel", cascade='all, delete-orphan'))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, ','.join(p.name.encode('ascii', errors='ignore') for p in self.persons))

    
DeclarativeBase.metadata.create_all(engine)

In [48]:
try:
    # Sadly in this case it won't work. SQLAlchemy is unable to
    # understand who is the parent so it will refuse to work
    person1 = Person(name='person1')
except Exception as e:
    print e

On Person.manylogsdel, delete-orphan cascade is not supported on a many-to-many or many-to-one relationship when single_parent is not set.   Set single_parent=True on the relationship().
